In [1]:
library(rtracklayer)

Loading required package: GenomicRanges

Loading required package: stats4

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which, which.max, which.min


Loading required package: S4Vecto

In [2]:
library(Biostrings)

Loading required package: XVector


Attaching package: ‘Biostrings’


The following object is masked from ‘package:base’:

    strsplit




In [16]:
library(data.table)


Attaching package: ‘data.table’


The following object is masked from ‘package:GenomicRanges’:

    shift


The following object is masked from ‘package:IRanges’:

    shift


The following objects are masked from ‘package:S4Vectors’:

    first, second




In [199]:
genomes_avail <- ucscGenomes()

In [227]:
test_run_conc <- read.csv("../meta/validation_species.csv", sep = ";")

In [228]:
test_run_conc <- test_run_conc[test_run_conc$species %in% c("Sheep", "Chicken", "Elephant shark", "Tasmanian devil", "Wallaby"),]

In [229]:
test_run_conc

,species,ucsc_genome,class,scientific_name
,<fct>,<fct>,<fct>,<fct>
7,Elephant shark,calMil1,Chondrichthyes,Callorhinchus milii
19,Chicken,galGal6,Aves,Gallus gallus
24,Tasmanian devil,sarHar1,Marsupiala,Sarcophilus harrisii
42,Sheep,oviAri4,Mammalia,Ovis aries
47,Wallaby,macEug2,Marsupiala,Notamacropus eugenii


In [230]:
a <- as.character(test_run_conc$scientific_name[[1]])

In [231]:
b <- strsplit(a, " ")[[1]]

In [217]:
paste0(substr(b[1],1,1), b[2], collapse = "")

[1] "Cmilii"

In [232]:
test_run_conc$genome_id <- sapply(test_run_conc$scientific_name, function(x) {b <- strsplit(as.character(x), " ")[[1]]; 
                                                                              id <- paste0(substr(b[1],1,1), b[2], collapse = ""); return(id)} )

In [233]:
test_run_conc

,species,ucsc_genome,class,scientific_name,genome_id
,<fct>,<fct>,<fct>,<fct>,<chr>
7,Elephant shark,calMil1,Chondrichthyes,Callorhinchus milii,Cmilii
19,Chicken,galGal6,Aves,Gallus gallus,Ggallus
24,Tasmanian devil,sarHar1,Marsupiala,Sarcophilus harrisii,Sharrisii
42,Sheep,oviAri4,Mammalia,Ovis aries,Oaries
47,Wallaby,macEug2,Marsupiala,Notamacropus eugenii,Neugenii


In [234]:
write.table(test_run_conc, "../meta/genomes_to_concatinate.txt", sep = "\t", quote = F, row.names = F)

In [225]:
file.exists("/nobackup/lab_bock/projects/DNAmeth500species/resources/reference_genomes/genomes/Oaries/oviAri4.fa")

[1] TRUE

## download

In [28]:
cmd = paste0("wget http://hgdownload.soe.ucsc.edu/goldenPath/oviAri4/bigZips/oviAri4.fa.gz  -P /nobackup/lab_bock/projects/DNAmeth500species/validation/insilicoDigest/data/genomes/Oaries")

In [29]:
system(cmd)

In [32]:
cmd = paste0("gunzip /nobackup/lab_bock/projects/DNAmeth500species/validation/insilicoDigest/data/genomes/Oaries/oviAri4.fa.gz")

In [33]:
system(cmd)

In [36]:
cmd = paste0("wget http://hgdownload.soe.ucsc.edu/goldenPath/oviAri4/bigZips/oviAri4.chrom.sizes  -P /nobackup/lab_bock/projects/DNAmeth500species/validation/insilicoDigest/data/genomes/Oaries/")

In [37]:
system(cmd)

## upload

In [4]:
chrom_sizes = read.csv("/nobackup/lab_bock/projects/DNAmeth500species/validation/insilicoDigest/data/genomes/Oaries/oviAri4.chrom.sizes", sep = "\t", header = F)

In [5]:
tile_size = round(sum(chrom_sizes$V2)/20)

In [6]:
fa_file <- readDNAStringSet("/nobackup/lab_bock/projects/DNAmeth500species/validation/insilicoDigest/data/genomes/Oaries/oviAri4.fa")

In [7]:
head(fa_file)

  A DNAStringSet instance of length 6
        width seq                                           names               
[1] 275406953 GATAAAAAATAAATAGAAACA...GTGTCTTCCAACCATCTTCCT chr1
[2]  86377204 GTCTCCCCAGGGGAGTCAGTC...GGTTAGGGTTAGGGTTAGGGT chr10
[3]  62170480 GCCGGGGCCTAAGACTTTGTG...ATTGGAAGGACTGGTTTCCCC chr11
[4]  79028859 GATCATAAGAGTATTTGCCTT...CCCGCCGGCCCGCACCCCCCT chr12
[5]  82951069 CGTCATATCTGTAAAATAGAA...TTGAAAGTGGCGGGCACAGTA chr13
[6]  62568341 AGGTTTCTCTGGAATGGACTG...GTGTGAGTTCTCTGTGGCTTC chr14

In [221]:
genome_id <- "Oaries"

In [222]:
paste0("../../resources/reference_genomes/genomes/", genome_id, "/")

[1] "../../resources/reference_genomes/genomes/Oaries/"

In [8]:
NROW(fa_file)

[1] 5466

In [21]:
setDT(chrom_sizes)

In [12]:
sort(chrom_sizes$width)

NULL

In [26]:
head(chrom_sizes)

V1,V2
<fct>,<int>
chr1,275406953
chr2,248966461
chr3,223996068
chrX,135185801
chr4,119216639
chr6,116888256


In [25]:
chrom_sizes <- chrom_sizes[order(V2, decreasing = T)]

In [35]:
chrom_leave <- chrom_sizes[V2>0.8*tile_size]

In [39]:
setDT(chrom_leave)

In [185]:
##first saving the first one (no append=True on purpose!):
cat(">chrArt1", file = "../../resources/reference_genomes/genomes/Oaries/oviAri4_concatinated.fa", sep = "\n")

In [186]:
cat(as.character(fa_file[[chrom_sizes$V1[[1]]]]), file = "../../resources/reference_genomes/genomes/Oaries/oviAri4_concatinated.fa", sep = "\n", append = T)

In [187]:
chrom.map = data.table(new = "chrArt1", old = as.character(chrom_sizes$V1[[1]]), width_old = chrom_sizes$V2[[1]], width_new = chrom_sizes$V2[[1]])

In [188]:
for(chrom_id in c(2:NROW(chrom_sizes[V2>0.8*tile_size]))){
    print(chrom_id)
   cat(paste0(">chrArt",chrom_id), file = "../../resources/reference_genomes/genomes/Oaries/oviAri4_concatinated.fa", sep = "\n", append = T) 
    cat(as.character(fa_file[[chrom_leave$V1[[chrom_id]]]]), file = "../../resources/reference_genomes/genomes/Oaries/oviAri4_concatinated.fa", sep = "\n", append = T)
     chrom.map <- rbind(chrom.map, list("new" = paste0("chrArt",chrom_id), "old" = as.character(chrom_leave$V1[[chrom_id]])))
}

[1] 2
[1] 3
[1] 4
[1] 5
[1] 6
[1] 7


In [189]:
chrom.map

new,old
<chr>,<chr>
chrArt1,chr1
chrArt2,chr2
chrArt3,chr3
chrArt4,chrX
chrArt5,chr4
chrArt6,chr6
chrArt7,chr5


In [190]:
chrom_sizes[chrom_id + 1]$V1

[1] chr7
5466 Levels: chr1 chr10 chr11 chr12 chr13 chr14 chr15 chr16 chr17 chr18 ... chrX

In [191]:
chrom_id = chrom_id + 1
j = chrom_id ##first row of the leftover

print(j)
while(j < NROW(chrom_sizes)){
    i = j ## getting the moving count
    used_length = chrom_sizes[i]$V2
    print(as.character(chrom_sizes[i]$V1))
    while(used_length < tile_size && i  < NROW(chrom_sizes)){
        i = i+1
        used_length = used_length + chrom_sizes[i]$V2 ## next chromosome to add
    }
    chrom.map <- rbind(chrom.map, list("new" = paste0("chrArt",chrom_id), "old" = paste(as.character(chrom_sizes[c(j:i)]$V1), collapse = ";")))
    cat(paste0(">chrArt",chrom_id), file = "../../resources/reference_genomes/genomes/Oaries/oviAri4_concatinated.fa", sep = "\n", append = T) 
    cat(paste(as.character(fa_file[chrom_sizes[c(j:i)]$V1]), collapse = linker), file = "../../resources/reference_genomes/genomes/Oaries/oviAri4_concatinated.fa", sep = "\n", append = T)
    j = i + 1 ## moving on to the next chromosome(that is not included yet)
    print(j)
    chrom_id <- chrom_id + 1  ## starting the new artificial chromosome
}

[1] 8
[1] "chr7"
[1] 10
[1] "chr8"
[1] 12
[1] "chr13"
[1] 14
[1] "chr12"
[1] 16
[1] "chr16"
[1] 18
[1] "chr14"
[1] 21
[1] "chr19"
[1] 24
[1] "chr21"
[1] 27
[1] "chr24"
[1] 5467


In [192]:
chrom_sizes[8]

V1,V2
<fct>,<int>
chr7,100009711


In [193]:
head(chrom.map, n = 15)

new,old
<chr>,<chr>
chrArt1,chr1
chrArt2,chr2
chrArt3,chr3
chrArt4,chrX
chrArt5,chr4
chrArt6,chr6
chrArt7,chr5
chrArt8,chr7;chr9
chrArt9,chr8;chr10


In [197]:
write.table(chrom.map, "../../resources/reference_genomes/genomes//Oaries/oviAri4_chrom_mapping.tsv", sep = "\t", row.names = F, quote = F)

In [63]:
linker = paste(rep("N", 100), collapse = "")